<a href="https://colab.research.google.com/github/NishantTiwari00786/CS170-Puzzle-Problem/blob/main/CS170_Project_8_Side_Puzzle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [118]:
%%writefile puzzle.cpp
#include <iostream>
#include <utility> // Include for std::pair
#include <vector>
#include <queue>
#include <unordered_set>
#include <algorithm>
#include <string>

using namespace std;


// In the below code, all the source sthat I have used are explaind in detail along with their respecitve URL

// https://www.geeksforgeeks.org/uniform-cost-search-ucs-in-ai/
// https://stackoverflow.com/questions/42877601/array-of-nodes-initialization
//

struct Node {
    int puzzle[3][3];
    int gCost;
    int hCost;
    int fCost;
    Node* parent;

    // Updated Constructor
    Node(const int (puzzle)[3][3], int gCost, Node* pr = nullptr)
        : gCost(gCost), hCost(0), fCost(gCost), parent(pr) {

        // Copy the puzzle state
        for (int i = 0; i < 3; i++) {
            for (int j = 0; j < 3; j++) {
                this->puzzle[i][j] = puzzle[i][j];
            }
        }
    }

    // Function to update heuristic and fCost after creating a node
   void updateCosts(int (*heuristic)(const int[3][3])) {
    this->hCost = heuristic(this->puzzle);
    this->fCost = this->gCost + this->hCost;
}
};

// using std::priority func in c++ to find the shortest Node

// this struct is used in uniform cost search
struct CompareNodes{
  bool operator()(const Node* a, const Node* b) const {
    return a->gCost > b->gCost;
  }
};

// this struct is used in astar with misplaced and manhattan

// https://stackoverflow.com/questions/14981590/priority-queue-declaration-and-bool-operator-declaration - used this as a reference to get the syntax hint
struct CompareNodesAstar{
  bool operator()(const Node* a, const Node* b) const {
    return a->fCost > b->fCost;
  }
};



// converted the puzzle to string, used it as a helper to keep track of all the states that are visited

// https://stackoverflow.com/questions/14981590/priority-queue-declaration-and-bool-operator-declaration - used this as a reference to get the syntax hint
string puzzle_to_string(int puzzle[3][3]) {
  string result;
  for (int i = 0; i < 3; i++) {
    for (int j = 0; j < 3; j++) {
      result += std::to_string(puzzle[i][j]);
    }

  }
  return result;
}


// Check for the goal state for 8 side puzzle, to be called again, to check if goal state has reached or not.
// the link below gave me an idea, that we need to have a function that can keep the check if goal state is reached or not
// https://www.almabetter.com/bytes/tutorials/artificial-intelligence/8-puzzle-problem-in-ai

bool isGoalState(int puzzle[3][3]) {
    int goal[3][3] = {
        {1, 2, 3},
        {4, 5, 6},
        {7, 8, 0}
    };

    for (int i = 0; i < 3; i++) {
        for (int j = 0; j < 3; j++) {
            if (puzzle[i][j] != goal[i][j]) {
                return false;
            }
        }
    }
    return true;
}


// simple function to print the puzzle in each step to keep the trace of the program while it is running
void print_puzzle(int puzzle[3][3]) {
    cout << "Testing Puzzle state:" << endl;
    for (int i = 0; i < 3; i++) {
        for (int j = 0; j < 3; j++) {
            cout << puzzle[i][j] << " ";
        }
        cout << endl;
    }
}

// simple function to print the step no: and dkeep track of depth

void print_solution(Node* sol) {
    vector<Node*> path;
    int depth = 0;  // Initialize depth

    while (sol != nullptr) {
        path.push_back(sol);
        sol = sol->parent;
    }

    reverse(path.begin(), path.end());

    for (Node* node : path) {
        cout << "Step " << depth++ << " (Depth: " << node->gCost << "):" << endl;
        print_puzzle(node->puzzle);
        cout << "-----------------" << endl;
    }

    cout << "Total Depth of Solution: " << depth - 1 << endl;
}


// https://stackoverflow.com/questions/63759863/c-how-to-find-pair-of-3-integers-in-vector-pairint-pairint - used this stackoverflow link to reiterate how what grid with blank value only used to understand functionality

pair<int, int> findBlankTile(int puzzle[3][3]) {
    for (int i = 0; i < 3; i++) {
        for (int j = 0; j < 3; j++) {
            if (puzzle[i][j] == 0) {
                return make_pair(i, j);
            }
        }
    }
    return make_pair(-1, -1); // Indicate invalid puzzle
}

// copy puzzle for transition - used when to switch from one state to other
void copyPuzzle(int src[3][3], int dest[3][3]) {
  for(int i = 0; i < 3; i++) {
    for(int j = 0; j < 3; j++) {
      dest[i][j] = src[i][j];
    }
  }
}

// https://www.geeksforgeeks.org/8-puzzle-problem-using-branch-and-bound/

// Used this link for moveLeft and then did rest by myself https://stackoverflow.com/questions/13187289/moving-elements-in-a-3x3-array-based-on-user-inputchoice-of-up-down-left-or

void generating_Moves(int puzzle[3][3]) {

    pair<int, int> blankTile = findBlankTile(puzzle);
    int row = blankTile.first;
    int col = blankTile.second;

    for (int i = 0; i < 3; i++) {


    if (row > 0){

      int puzzleCopy[3][3];

      copyPuzzle(puzzle, puzzleCopy);

      puzzleCopy[row][col] = puzzleCopy[row - 1][col];
      puzzleCopy[row - 1][col] = 0;
      print_puzzle(puzzleCopy);

      cout << "Move up" << endl;

    }


    if (row < 2) {
      int puzzleCopy[3][3];

      copyPuzzle(puzzle, puzzleCopy);
      puzzleCopy[row][col] = puzzleCopy[row + 1][col];
      puzzleCopy[row + 1][col] = 0;
      print_puzzle(puzzleCopy);

      cout << "Move down" << endl;
    }

    if (col > 0) {
      int puzzleCopy[3][3];

      copyPuzzle(puzzle, puzzleCopy);
      puzzleCopy[row][col] = puzzleCopy[row][col - 1];
      puzzleCopy[row][col - 1] = 0;
      print_puzzle(puzzleCopy);

      cout << "Move left" << endl;
    }

    if (col < 2) {
      int puzzleCopy[3][3];

      copyPuzzle(puzzle, puzzleCopy);
      puzzleCopy[row][col] = puzzleCopy[row][col + 1];
      puzzleCopy[row][col + 1] = 0;
      print_puzzle(puzzleCopy);


      cout << "Move right" << endl;

      }
}
}

// simple count function to see how many tiles are misplaced - it goes over the puzzle, compares it to goal state and print the count which tells how off it is  - used Dr. Keogh CS170 Chapter 4-5 slides to understand the logic behind it

int misplaced_HeurisitcTile(const int puzzle[3][3]) {
  int goal_state[3][3] = {
        {1, 2, 3},
        {4, 5, 6},
        {7, 8, 0}
    };
    int count = 0;
    for (int i = 0; i < 3; i++) {
        for (int j = 0; j < 3; j++) {
            if (puzzle[i][j]!=0 && puzzle[i][j] !=  goal_state[i][j]) {
                count++;
            }
        }
    }
    return count;
}

// simple function for heursisitc of Manhattan distance - similar explanation like above again referred to slides and https://stackoverflow.com/questions/39759721/calculating-the-manhattan-distance-in-the-eight-puzzle - used this link to understand math behind manhattan distance

int manhattanDistance_Heurisitc(const int puzzle[3][3]) {

  int diff_distance = 0;

  for (int i = 0; i < 3; i++) {
    for (int j = 0; j < 3; j++) {

      int mis_tile = puzzle[i][j];
      if (mis_tile != 0) {
        int goal_row = (mis_tile - 1) / 3;
        int goal_col = (mis_tile - 1) % 3;
        diff_distance += abs(i - goal_row) + abs(j - goal_col);
      }
    }

  }
  return diff_distance;
}




// https://www.geeksforgeeks.org/8-puzzle-problem-using-branch-and-bound/ - referred the solve function from that particular given link - this function is specific to UCS, because it only accounts for gCost
// https://www.geeksforgeeks.org/how-to-access-vector-methods-from-pointer-to-vector-in-cpp/ - used this link for refreshing my memory towards using vector with pointers, rest of the work is original

vector<Node*> checkValidMoves(Node* current) {
    vector<Node*> children;

    int Total_cost = current->gCost + 1; // because g(n) is 1, as shown in prof. slides

    pair<int, int> blankTile = findBlankTile(current->puzzle);
    int row = blankTile.first;
    int col = blankTile.second;

    static const int dx[] = {-1, 1, 0, 0};
    static const int dy[] = {0, 0, -1, 1};

    for (int i = 0; i < 4; i++) {
        int newRow = row + dx[i];
        int newCol = col + dy[i];
        if (newRow >= 0 && newRow < 3 && newCol >= 0 && newCol < 3) {

            int puzzleCopy[3][3];
            copyPuzzle(current->puzzle, puzzleCopy);
            swap(puzzleCopy[row][col], puzzleCopy[newRow][newCol]);

            Node* childptr = new Node(puzzleCopy, Total_cost, current);
            children.push_back(childptr);
        }
    }

    return children;
}

// https://www.geeksforgeeks.org/8-puzzle-problem-using-branch-and-bound/ - referred the solve function from that particular given link - this function is specific to UCS, because it only accounts for gCost
// https://www.geeksforgeeks.org/how-to-access-vector-methods-from-pointer-to-vector-in-cpp/ - used this link for refreshing my memory towards using vector with pointers, rest of the work is original


vector<Node*> checkValidMoves_Heuristic(Node* current, int (*heuristic)(const int[3][3])) {
    vector<Node*> children;
    int new_gCost = current->gCost + 1;

    pair<int, int> blankTile = findBlankTile(current->puzzle);
    int row = blankTile.first;
    int col = blankTile.second;

    static const int dx[] = {-1, 1, 0, 0};
    static const int dy[] = {0, 0, -1, 1};

    for (int i = 0; i < 4; i++) {
        int newRow = row + dx[i];
        int newCol = col + dy[i];

        if (newRow >= 0 && newRow < 3 && newCol >= 0 && newCol < 3) {
            int puzzleCopy[3][3];
            copyPuzzle(current->puzzle, puzzleCopy);
            swap(puzzleCopy[row][col], puzzleCopy[newRow][newCol]);

            Node* childptr = new Node(puzzleCopy, new_gCost, current);
            childptr->updateCosts(heuristic);

            children.push_back(childptr);
        }
    }
    return children;
}



// https://www.almabetter.com/bytes/tutorials/artificial-intelligence/8-puzzle-problem-in-ai - used this link to understand on how to convert math to code (coding part is all original, only used the link to understand how to tackle the problem)
// https://www.geeksforgeeks.org/how-to-access-vector-methods-from-pointer-to-vector-in-cpp/  - used this again for understanding the syntax as done in previous func
Node *uniformCostSearch(int puzzle[3][3]) {

    Node* root = new Node(puzzle, 0, nullptr);

    //check if it is already in the goal state
    if(isGoalState(root->puzzle)) {
        return root;
    }

    // starting with the one with least path
    priority_queue<Node*, vector<Node*>, CompareNodes> pq;
    pq.push(root);

    // used to track the visited states
    unordered_set<string> visited;
    visited.insert(puzzle_to_string(root->puzzle));

    while(!pq.empty()) {
        Node* current = pq.top();
        pq.pop();

        if(isGoalState(current->puzzle)) {
            return current;
        }

        vector<Node*> children = checkValidMoves(current);

        for(Node* child : children) {
            string childState = puzzle_to_string(child->puzzle);
            if(visited.find(childState) == visited.end()) {
                visited.insert(childState);
                pq.push(child);
            }
        }
    }

    return nullptr;
}

// same links as used in *uniform cost search
Node *aStarSearch(int puzzle[3][3]) {

  Node* root = new Node(puzzle, 0, nullptr);

  //check if it is already in the goal state
  if(isGoalState(root->puzzle)) {
      return root;
  }

  // starting with the one with least path
  priority_queue<Node*, vector<Node*>, CompareNodesAstar> pq;
  pq.push(root);

  // used to track the visited states
  unordered_set<string> visited;
  visited.insert(puzzle_to_string(root->puzzle));

  while(!pq.empty()) {
      Node* current = pq.top();
      pq.pop();

      if(isGoalState(current->puzzle)) {
          return current;
      }

      vector<Node*> children = checkValidMoves_Heuristic(current, misplaced_HeurisitcTile);

      // calling the misplaced tile heuristic here - wrote it as a debugging statement
      for(Node* child : children) {
         child -> hCost = misplaced_HeurisitcTile(child->puzzle);
         child -> fCost = child -> gCost + child -> hCost;

          string childState = puzzle_to_string(child->puzzle);
          if(visited.find(childState) == visited.end()) {
              visited.insert(childState);
              pq.push(child);
          }
      }
  }
  return nullptr;
}


// same links as used in *uniform cost search

Node *astar_manhattan(int puzzle[3][3]) {

  Node* root = new Node(puzzle, 0, nullptr);
  root -> updateCosts(manhattanDistance_Heurisitc);

  root -> hCost = manhattanDistance_Heurisitc(root->puzzle);
  root -> fCost = root -> gCost + root -> hCost;

  if(isGoalState(root->puzzle)) {
      return root;
  }

  priority_queue<Node*, vector<Node*>, CompareNodesAstar> pq;
  pq.push(root);

  unordered_set<string> visited;
  visited.insert(puzzle_to_string(root->puzzle));

  while(!pq.empty()) {
      Node* current = pq.top();
      pq.pop();

      if(isGoalState(current->puzzle)) {
          return current;
      }

      vector<Node*> children = checkValidMoves_Heuristic(current, manhattanDistance_Heurisitc);

      for (Node* child: children){

        // wrote it as a debugging statement

        child -> hCost = manhattanDistance_Heurisitc(child->puzzle);
        child -> fCost = child -> gCost + child -> hCost;

        string childState = puzzle_to_string(child->puzzle);
        if(visited.find(childState) == visited.end()) {
            visited.insert(childState);
            pq.push(child);
        }
      }
      }
      return nullptr;
  }

// referred to my previous CS100 - Chess game assignment for main layout - https://github.com/NishantTiwari00786/Command-Line-Chess-Game
// I tried my best to mimic the given sample main in assignment, but mine doesn't print max queueu nods, it only prints depth - I noticed it while I was finalizng my report that we do have to print other stuff too, but for that I had to make some changes to other functions which I resorted not too
// For this you can only select one algorithm in one run, and then restart the program again for other puzzles, but in report i have listed all the algorithms working
int main() {
    int defaultPuzzle[3][3] = {
        {1, 3, 6},
        {5, 0, 2},
        {4, 7, 8}  // default puzzle if user wants presses option 1
    };
    int customPuzzle[3][3];

    cout << "Welcome to my 8-Puzzle Solver. Type '1' to use a default puzzle, or '2' to create your own.\n";
    int choice;
    cin >> choice;

    if (choice == 1) {
        cout << "Using default puzzle.\n";
    } else if (choice == 2) {
        cout << "Enter your puzzle, using a zero to represent the blank. Enter each row's numbers separated by space. Press ENTER after each row:\n";
        for (int i = 0; i < 3; i++) {
            cout << "Enter the " << (i + 1) << " row: ";
            for (int j = 0; j < 3; j++) {
                cin >> customPuzzle[i][j];
            }
            cout << "Row " << (i + 1) << " added.\n";
        }
    } else {
        cout << "Invalid choice. Please restart the program and select '1' or '2'.\n";
        return 1;
    }

    int (*puzzle)[3][3] = (choice == 1) ? &defaultPuzzle : &customPuzzle;

    cout << "Initial Puzzle State:\n";
    print_puzzle(*puzzle);
    cout << "------------------------------\n";

    while (true) {
        cout << "Select algorithm. (1) for Uniform Cost Search, (2) for the Misplaced Tile Heuristic, or (3) for the Manhattan Distance Heuristic, (0) to Exit.\n";
        int user_choice;
        cin >> user_choice;

        if (user_choice == 0) {
            cout << "Exiting program.\n";
            break;
        }

        Node* result = nullptr;
        switch (user_choice) {
            case 1:
                cout << "Testing Uniform Cost Search (UCS)...\n";
                result = uniformCostSearch(*puzzle);
                break;
            case 2:
                cout << "Testing A* Search (Misplaced Tile Heuristic)...\n";
                result = aStarSearch(*puzzle);
                break;
            case 3:
                cout << "Testing A* Search (Manhattan Distance Heuristic)...\n";
                result = astar_manhattan(*puzzle);
                break;
            default:
                cout << "Invalid algorithm choice.\n";
                continue;
        }

        if (result) {
            cout << "Solution Found:\n";
            print_solution(result);
        } else {
            cout << "No Solution Found.\n";
        }
        cout << "------------------------------\n";
    }

    return 0;
}

Overwriting puzzle.cpp


In [119]:
!g++ puzzle.cpp -o puzzle

In [120]:
!./puzzle

Welcome to my 8-Puzzle Solver. Type '1' to use a default puzzle, or '2' to create your own.
1
Using default puzzle.
Initial Puzzle State:
Testing Puzzle state:
1 3 6 
5 0 2 
4 7 8 
------------------------------
Select algorithm. (1) for Uniform Cost Search, (2) for the Misplaced Tile Heuristic, or (3) for the Manhattan Distance Heuristic, (0) to Exit.
2
Testing A* Search (Misplaced Tile Heuristic)...
Solution Found:
Step 0 (Depth: 0):
Testing Puzzle state:
1 3 6 
5 0 2 
4 7 8 
-----------------
Step 1 (Depth: 1):
Testing Puzzle state:
1 3 6 
5 2 0 
4 7 8 
-----------------
Step 2 (Depth: 2):
Testing Puzzle state:
1 3 0 
5 2 6 
4 7 8 
-----------------
Step 3 (Depth: 3):
Testing Puzzle state:
1 0 3 
5 2 6 
4 7 8 
-----------------
Step 4 (Depth: 4):
Testing Puzzle state:
1 2 3 
5 0 6 
4 7 8 
-----------------
Step 5 (Depth: 5):
Testing Puzzle state:
1 2 3 
0 5 6 
4 7 8 
-----------------
Step 6 (Depth: 6):
Testing Puzzle state:
1 2 3 
4 5 6 
0 7 8 
-----------------
Step 7 (Depth: 7):
